In [1]:
import numpy as np
from sklearn import metrics
from scipy.special import comb, perm
from scipy.stats import entropy
import inspect

## Rand Index

If C is a ground truth class assignment and K the clustering, let us define  and  as:

, the number of pairs of elements that are in the same set in C and in the same set in K

, the number of pairs of elements that are in different sets in C and in different sets in K

The unadjusted Rand index is then given by:

$$\mathrm{RI}=\frac{a+b}{C_{2}^{n_{\text {samples }}}}$$

$$\mathrm{ARI}=\frac{\mathrm{RI}-E[\mathrm{RI}]}{\max (\mathrm{RI})-E[\mathrm{RI}]}$$

优点:
1. 可解释性好: 数值代表了有多少对样本下, 聚类结果和标签是一致或者不一致的
2. 随机情况下均值为0: ARI在0附近波动
3. 有界: 取值为-1,1
4. 对聚类结果形状无要求

缺点:
1. 不调整的RI在类别数多的时候, 趋向于1
2. 需要知道真实的label

In [122]:
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 3, 3]

In [47]:
def contingency_table(result, label):
    
    total_num = len(label)
    
    TP = TN = FP = FN = 0
    for i in range(total_num):
        for j in range(i + 1, total_num):
            if label[i] == label[j] and result[i] == result[j]:
                TP += 1
            elif label[i] != label[j] and result[i] != result[j]:
                TN += 1
            elif label[i] != label[j] and result[i] == result[j]:
                FP += 1
            elif label[i] == label[j] and result[i] != result[j]:
                FN += 1
    return (TP, TN, FP, FN)
def rand_index(result, label):
    TP, TN, FP, FN = contingency_table(result, label)
    return 1.0*(TP + TN)/(TP + FP + FN + TN)

In [123]:
metrics.rand_score(labels_true, labels_pred),rand_index(labels_true, labels_pred)

(0.6666666666666666, 0.6666666666666666)

In [124]:
metrics.adjusted_rand_score(labels_true, labels_pred)

0.24242424242424243

In [128]:
np.append(np.random.choice(3, size = 3, replace = True), np.random.choice(3, size = 3, replace = True))

array([1, 2, 2, 2, 1, 1])

In [129]:
# Get Expectation of MI by simulation. This is not correct, see original paper: https://link.springer.com/article/10.1007/BF01908075
np.random.seed(12)
res = []
for i in range(1000):
    iter_pred = np.append(np.random.choice([0,1,2], size = 3, replace = True), np.random.choice([3,4,5], size = 3, replace = True))
    res.append(metrics.rand_score(labels_true, iter_pred))
# AMI
metrics.adjusted_rand_score(labels_true, labels_pred),\
(metrics.rand_score(labels_true, labels_pred) - np.mean(res))/(np.max(res) - np.mean(res))

(0.24242424242424243, -0.26646403242147887)

In [70]:
# K= 100, RI tends to be 1
labels_true = np.random.randint(0,100,size = 100)
labels_pred = np.random.randint(0,100,size = 100)
metrics.rand_score(labels_true, labels_pred), metrics.adjusted_rand_score(labels_true, labels_pred)

(0.9826262626262626, 0.014035575319622013)

## Mutual Information 
Assume two label assignments(of the same N objects), U and V. Their entropy is the amount of uncertainty for a partition set, define by:
$$H(U)=-\sum_{i=1}^{|U|} P(i) \log (P(i))$$
$$H(V)=-\sum_{j=1}^{|V|} P^{\prime}(j) \log \left(P^{\prime}(j)\right)$$
The mutual information(MI) between U and V is calculated by:
$$\operatorname{MI}(U, V)=\sum_{i=1}^{|U|} \sum_{j=1}^{|V|} P(i, j) \log \left(\frac{P(i, j)}{P(i) P^{\prime}(j)}\right)$$
also can be expressed in set cardinality formulation:
$$\operatorname{MI}(U, V)=\sum_{i=1}^{|U|} \sum_{j=1}^{|V|} \frac{\left|U_{i} \cap V_{j}\right|}{N} \log \left(\frac{N\left|U_{i} \cap V_{j}\right|}{\left|U_{i}\right|\left|V_{j}\right|}\right)$$


normlized mutual information is defined as:
$$\operatorname{NMI}(U, V)=\frac{\operatorname{MI}(U, V)}{\operatorname{mean}(H(U), H(V))}$$

adjusted mutual infromation is defined as:
$$\mathrm{AMI}=\frac{\mathrm{MI}-E[\mathrm{MI}]}{\operatorname{mean}(H(U), H(V))-E[\mathrm{MI}]}$$
where, 
$$E[\operatorname{MI}(U, V)]=\sum_{i=1}^{|U|} \sum_{j=1}^{|V|} \sum_{n_{i j}=\left(a_{i}+b_{j}-N\right)^{+}}^{\min \left(a_{i}, b_{j}\right)} \frac{n_{i j}}{N} \log \left(\frac{N \cdot n_{i j}}{a_{i} b_{j}}\right) \frac{a_{i} ! b_{j} !\left(N-a_{i}\right) !\left(N-b_{j}\right) !}{N ! n_{i j} !\left(a_{i}-n_{i j}\right) !\left(b_{j}-n_{i j}\right) !\left(N-a_{i}-b_{j}+n_{i j}\right) !}$$

优点:
1. 随机情况下, AMI接近与0
2. 有界: [-1,1]之间

缺点:
1. NMI和MI在随机情况下并不接近于0
2. 需要知道真实的label

In [77]:
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]

In [96]:
# self-define entroy function
def entropy1(labels, base=None):
    value,counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)

In [100]:
# MI
metrics.mutual_info_score(labels_true, labels_pred)

0.4620981203732969

In [99]:
# NMI
metrics.normalized_mutual_info_score(labels_true, labels_pred) ,\
metrics.mutual_info_score(labels_true, labels_pred)/np.mean([entropy1(labels_true), entropy1(labels_pred)])

(0.5158037429793889, 0.5158037429793888)

In [116]:
# Get Expectation of MI by simulation
np.random.seed(12)
res = []
for i in range(1000):
    iter_pred = np.random.choice(labels_pred, size = labels_pred.__len__(), replace = False)
    res.append(metrics.mutual_info_score(labels_true, iter_pred))
# AMI
metrics.adjusted_mutual_info_score(labels_true, labels_pred),\
(metrics.mutual_info_score(labels_true, labels_pred) - np.mean(res))/(np.mean([entropy1(labels_true), entropy1(labels_pred)]) - np.mean(res))

(0.2987924581708901, 0.29457695646633375)

## Homogeneity, completeness and V-measure
homogeneity: each cluster contains only members of a single class.

completeness: all members of a given class are assigned to the same cluster.

v_measure: harmonic mean of homogeneity and completness
$$v=\frac{(1+\beta) \times \text { homogeneity } \times \text { completeness }}{(\beta \times \text { homogeneity }+\text { completeness })}$$

优点:
1. 值有界: 都介于0和1之间
2. 可解释性强: 类似于召回和精准

缺点:
1. 随机情况下不为0. 样本量>1000, 类别<=10的时候可用.
2. 需要真实值.

In [132]:
metrics.homogeneity_score(labels_true, labels_pred),metrics.completeness_score(labels_true, labels_pred)

(0.6666666666666669, 0.420619835714305)

In [133]:
metrics.v_measure_score(labels_true, labels_pred)

0.5158037429793889